<a href="https://colab.research.google.com/github/AllamElsheikh/Ai_Journey-/blob/main/web_scrapping_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get data

In [1]:
!pip install pyppeteer
!pip install bs4
!pip install playwright
!playwright install
!pip install playwright

  Using cached pyee-11.1.1-py3-none-any.whl.metadata (2.8 kB)
Using cached pyee-11.1.1-py3-none-any.whl (15 kB)
  Attempting uninstall: pyee
    Found existing installation: pyee 12.0.0
    Uninstalling pyee-12.0.0:
      Successfully uninstalled pyee-12.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
playwright 1.47.0 requires pyee==12.0.0, but you have pyee 11.1.1 which is incompatible.
  Using cached pyee-12.0.0-py3-none-any.whl.metadata (2.8 kB)
Using cached pyee-12.0.0-py3-none-any.whl (14 kB)
  Attempting uninstall: pyee
    Found existing installation: pyee 11.1.1
    Uninstalling pyee-11.1.1:
      Successfully uninstalled pyee-11.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyppeteer 2.0.0 requires pyee<12.0.0,>=11.0

In [2]:
import os
from bs4 import BeautifulSoup as bsoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
from pyppeteer import launch

from pyppeteer.errors import TimeoutError as PyppeteerTimeoutError
import time

In [5]:
SEASONS = list(range(2022,2024))
SEASONS

[2022, 2023]

In [6]:
data_dir = "data"
standings_dir = os.path.join(data_dir , "standings")
scores = os.path.join(data_dir , "scores")

In [40]:
async def html_getter(url , selector, sleep = 4 , retries =3 ):
  html = None
  for i in range (1 ,retries+1):
    time.sleep(sleep * i)

    try :
      async with async_playwright() as p :
        browser = await p.firefox.launch()
        page = await browser.new_page()
        await page.goto(url)
        print(await page.title())
        html = await page.inner_html(selector)
    except PlaywrightTimeout:
      print(f"Timeout error on {url}")
      continue
    else :
      break
  return html


In [41]:
async def season_scrapper(seasson):
  url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
  html = await html_getter(url , "#content .filter")

  soup = bsoup(html)
  links = soup.find_all("a")
  hrefs = [l["href"] for l in links]
  standings_pages = [f"https://Basketball-Reference.com{l}" for l in hrefs]

  for url in standings_pages :
    save_path = os.path.join(standings_dir , url.split("/")[-1])
    if os.path.exists(save_path):
       continue

    html = await html_getter(url , "#all_schedule")
    with open(save_path , "w+") as f :
      f.write(html)

In [7]:
for season in SEASONS :
  await season_scrapper(season)

2019-20 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Refere

In [8]:
%ls data

scores/  standings/


In [7]:
standings_files = os.listdir(standings_dir )
standings_files

['NBA_2022_games-march.html',
 'NBA_2020_games-september.html',
 'NBA_2020_games-november.html',
 'NBA_2020_games-december.html',
 'NBA_2023_games-october.html',
 'NBA_2022_games-january.html',
 'NBA_2021_games-february.html',
 'NBA_2022_games-november.html',
 'NBA_2022_games-december.html',
 'NBA_2021_games-june.html',
 'NBA_2023_games-april.html',
 'NBA_2023_games-december.html',
 'NBA_2021_games-december.html',
 'NBA_2022_games-april.html',
 'NBA_2021_games-january.html',
 'NBA_2023_games-january.html',
 'NBA_2023_games-may.html',
 'NBA_2020_games-july.html',
 'NBA_2020_games-february.html',
 'NBA_2023_games-february.html',
 'NBA_2020_games-october.html',
 'NBA_2021_games-may.html',
 'NBA_2022_games-october.html',
 'NBA_2023_games-november.html',
 'NBA_2022_games-february.html',
 'NBA_2021_games-july.html',
 'NBA_2020_games-march.html',
 'NBA_2023_games-june.html',
 'NBA_2021_games-april.html',
 'NBA_2020_games-august.html',
 'NBA_2022_games-june.html',
 'NBA_2021_games-march.html',

In [71]:
async def game_scrapper(standings_file):

  with open(standings_file , "r") as f :
    html = f.read()

  soup = bsoup(html)
  links = soup.find_all("a")
  hrefs = [l.get("href") for l in links]
  box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
  box_scores = [f"https://Basketball-Reference.com{l}" for l in box_scores]

  for url in box_scores :
    save_path = os.path.join(scores , url.split("/")[-1])
    if os.path.exists(save_path):
      continue

    html = await html_getter(url , "#content")
    if not html :
      continue
    with open(save_path , "+w") as f :
      f.write(html)

In [53]:
standings_files = [s for s in standings_files if ".html" in s ]

In [ ]:
import pandas as pd

for season in SEASONS:
    files = [s for s in standings_files if str(season) in s]

    for f in files:
        filepath = os.path.join(standings_dir, f)

        await game_scrapper(filepath)

# Parsing data

In [73]:
box_scores = os.listdir(scores)

In [75]:
box_scores[:5]

['202203090MIL.html',
 '202203040PHI.html',
 '202203050MEM.html',
 '202203070DET.html',
 '202203030LAC.html']

In [ ]:
f.endwith

In [79]:
box_scores = [os.path.join(scores , f) for f in box_scores if f.endswith(".html")]

In [80]:
box_scores[:5]

['data/scores/202203090MIL.html',
 'data/scores/202203040PHI.html',
 'data/scores/202203050MEM.html',
 'data/scores/202203070DET.html',
 'data/scores/202203030LAC.html']

In [81]:
def parse_html(box_score):
  with open(box_score , "r") as f :
    html = f.read()

  soup = bsoup(html)
  [s.decompose() for s in soup.select("tr.over_header")]
  [s.decompose() for s in soup.select("tr.thead")]
  return soup

In [86]:
def read_line(soup):
    line_score = pd.read_html(str(soup), attrs = {'id': 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols

    line_score = line_score[["team", "total"]]

    return line_score

In [89]:
def read_stats(soup , team , stat) :
  df = pd.read_html(str(soup) , attrs = {"id" : f"box-{team}-game-{stat}"} , index_col = 0)[0]
  df = df.apply(pd.to_numeric , errors = "coerce")
  return df

In [93]:
box_score = box_scores[0]
soup = parse_html(box_score)
line_score = read_line(soup)
teams = list(line_score["team"])
summaries = []
for t in teams :
  basic = read_stats(soup , t , "basic")
  advanced = read_stats(soup , t , "advanced")
  total = pd.concat([basic.iloc[-1, :] , advanced.iloc[-1 , :]])
  maxes = pd.concat()

<ipython-input-86-faaf5f507331>:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs = {'id': 'line_score'})[0]
<ipython-input-89-ac0d235af43c>:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup) , attrs = {"id" : f"box-{team}-game-{stat}"} , index_col = 0)[0]
<ipython-input-89-ac0d235af43c>:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup) , attrs = {"id" : f"box-{team}-game-{stat}"} , index_col = 0)[0]
<ipython-input-89-ac0d235af43c>:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a f

In [ ]:
games = []
base_cols = None
for box_score in box_scores:
    soup = parse_html(box_score)

    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])

        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]

        summary = summary[base_cols]

        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0,1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)
    full_game["season"] = read_season_info(soup)

    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")

    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)

    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

In [ ]:
games_df = pd.concat(games, ignore_index=True)
games_df


In [ ]:
games_df.to_csv("nba_games.csv")
